In [18]:
import os

import librosa
import torch
import torch.nn as nn
import glob
import soundfile as sf

from academicodec.models.ticodec.vqvae import VQVAE
import json
from pathlib import Path

In [19]:
ckpt_path = '/home/jupyter/TiCodec/egs/TiCodec-24k-320d/logs_conv_only/logs_convonly_Lib_1g1r_cos_from_head/g_00190000'
config_path = '/home/jupyter/TiCodec/egs/TiCodec-24k-320d/logs_conv_only/logs_convonly_Lib_1g1r_cos_from_head/config.json'

In [20]:
with open(config_path, 'r') as f:
    config = json.load(f)
    sample_rate = config['sampling_rate']

outputdir = './output'
inputdir = './test_wav'

In [21]:
Path(outputdir).mkdir(parents=True, exist_ok=True)

In [22]:
model = VQVAE(config_path, ckpt_path, with_encoder = True)
model.generator.remove_weight_norm()
model.encoder.remove_weight_norm()
model.eval()
print("Model Ready")

/home/jupyter/TiCodec/academicodec/models/ticodec/vqvae.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path)
/opt/conda/envs/ticodec/lib/python3

Removing weight norm...
Removing weight norm...
Model Ready


In [23]:
wav_paths = glob.glob(f"{inputdir}/*.wav")
print(f"Globbed {len(wav_paths)} wav files.")

Globbed 5 wav files.


In [ ]:
frame_tokens = []
ti_tokens = []

for wav_path in wav_paths:
    wav, sr = librosa.load(wav_path, sr=sample_rate)
    fid = os.path.basename(wav_path)[:-4]
    wav = torch.tensor(wav).unsqueeze(0)
    wav = wav
    #vq_codes represent frame-level tokens
    #global_tokens represent time invariant tokens
    vq_codes, global_token = model.encode(wav)
    frame_tokens.append(vq_codes)
    ti_tokens.append(global_token)
    
    # calls the generator to reconstruct audio
    syn = model(vq_codes, global_token)
    wav = syn.squeeze().detach().numpy()
    sf.write(
        os.path.join(outputdir, f'{fid}.wav'), wav, sample_rate)


In [29]:
frame_tokens

[tensor([[[ 783],
          [ 309],
          [ 309],
          [ 842],
          [ 914],
          [ 914],
          [ 783],
          [ 783],
          [ 914],
          [ 217],
          [ 217],
          [ 217],
          [ 217],
          [ 914],
          [ 914],
          [ 914],
          [ 309],
          [ 646],
          [ 646],
          [ 309],
          [ 309],
          [ 646],
          [ 646],
          [  58],
          [  69],
          [1012],
          [ 444],
          [ 838],
          [ 380],
          [ 444],
          [ 590],
          [ 358],
          [ 201],
          [ 444],
          [ 842],
          [ 277],
          [ 295],
          [ 808],
          [ 808],
          [ 295],
          [ 380],
          [  35],
          [  35],
          [ 646],
          [ 746],
          [ 619],
          [  83],
          [ 590],
          [ 380],
          [ 746],
          [ 309],
          [  69],
          [  69],
          [ 649],
          [ 649],
          

In [30]:
ti_tokens

[tensor([[[819, 129, 647, 156, 986, 130,  14, 755]]]),
 tensor([[[819, 129, 647, 432, 986, 130,  14,   8]]]),
 tensor([[[784, 129, 647, 385, 986, 130,  14, 941]]]),
 tensor([[[784, 129, 647, 778, 612, 130,  14, 328]]]),
 tensor([[[819, 129, 647, 151, 612, 130,  14, 347]]])]